## IBGE Data Analysis Project

IBGE is the main source for all kinds os statistical data in Brazil. This repository is an exploration of interesting socioeconomic data as well as concepts like Data visualization, REST API abstraction and Test driven development applied to Data Science.

In [153]:
import urllib3
#import json
import matplotlib.pyplot as plot
import numpy
import pandas
from ibge_lib import *


In [154]:
http = urllib3.PoolManager()
http_fn = lambda url: http.request('GET', url)

aggrs_query_url = aggregates_url()
researches = query_ibge(aggrs_query_url, http_fn)

 ### Picking an aggregate

 IBGE's API needs the exact data aggregate code in order to supply any useful data. Therefore, we need to ask for the list of aggregates and search for the code of the aggregate we want. Luckly, these are grouped by research. In this case, we want the National Household Sampling Research (PNAD).

In [155]:
format_fn = lambda x: f"{x['id']}: {x['nome']}"
#list(map(format_fn, researches))

In [156]:
pnad_id = 'B5'
filter_pnad_fn = lambda x: x['id'] == pnad_id
pnad_metadata = next(filter(filter_pnad_fn, researches)) # 'next' gets the first element, in this case
#pnad_data

In [157]:
filtered_aggr = filter_aggregates(pnad_metadata, 'rendimento domiciliar per capita')
format_fn = lambda aggr: f"{aggr['id']}: {aggr['nome']}"
#list(map(format_fn, filtered_aggr))

In [158]:
income_bounds_aggr = 7438
cumm_income_dist_aggr = 7530
avg_income_per_class_aggr = 7532

In [159]:
income_bounds_metadata = query_ibge( metadata_url(income_bounds_aggr), http_fn )
#income_bounds_metadata['variaveis']

In [160]:
income_boounds_var = '10769'
#income_bounds_metadata['classificacoes']

In [161]:
income_bounds_classif = income_bounds_metadata['classificacoes'][0]['id']
income_bounds_classes = [ str(x['id']) for x in income_bounds_metadata['classificacoes'][0]['categorias'] ]
income_bounds_classes_fmt = f'[{",".join(income_bounds_classes)}]'
income_bounds_classes_fmt = f'{income_bounds_classif}{income_bounds_classes_fmt}'
#income_bounds_classes_fmt

In [162]:
var_query_url = variables_url( income_bounds_aggr, [income_boounds_var], [income_bounds_classes_fmt])
income_bounds_response = query_ibge(var_query_url, http_fn )
#income_bounds_response

In [163]:
income_bounds_results = income_bounds_response[0]['resultados'][1:]
income_bounds_serii = map( (lambda x: x['series'][0]['serie']) , income_bounds_results)
income_bounds_serii = list(income_bounds_serii)
#income_bounds_serii

In [164]:
income_percentiles = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99]
income_bound_columns = list(map( (lambda x: f'P{x}'), income_percentiles ))
years_queried = list(income_bounds_serii[0].keys())
#years_queried

In [165]:
series_to_row_fn = lambda series: list(map(float, series.values())) # Note: the 'int' refers to the integer constructor
income_bounds_data = list(map( series_to_row_fn , income_bounds_serii ))
income_bounds_data = numpy.matrix(income_bounds_data).transpose()
income_bounds_data = pandas.DataFrame( income_bounds_data , index=years_queried, columns=income_bound_columns )
income_bounds_data

,P5,P10,P20,P30,P40,P50,P60,P70,P80,P90,P95,P99
2015,123.0,198.0,306.0,411.0,529.0,672.0,814.0,1029.0,1380.0,2177.0,3353.0,7758.0
2016,114.0,197.0,307.0,433.0,552.0,696.0,875.0,1082.0,1479.0,2382.0,3714.0,8658.0
2017,102.0,199.0,316.0,451.0,578.0,730.0,933.0,1130.0,1511.0,2453.0,3821.0,9049.0
2018,103.0,201.0,334.0,473.0,610.0,773.0,967.0,1211.0,1641.0,2625.0,4067.0,9893.0
2019,107.0,211.0,348.0,498.0,650.0,826.0,1002.0,1282.0,1733.0,2723.0,4209.0,10365.0
2020,146.0,252.0,396.0,527.0,673.0,836.0,1035.0,1263.0,1682.0,2646.0,4082.0,9832.0


### Cumulative Income Distribution

In [166]:
cumm_income_dist_metadata = query_ibge( metadata_url(cumm_income_dist_aggr), http_fn )
#cumm_income_dist_metadata['variaveis']

In [167]:
cumm_income_dist_var = '10826'
#cumm_income_dist_metadata['classificacoes']

In [168]:
cumm_income_dist_classif = cumm_income_dist_metadata['classificacoes'][0]['id']
cumm_income_dist_classes = cumm_income_dist_metadata['classificacoes'][0]['categorias'][:-1]
cumm_income_dist_classes = map( lambda x: str(x['id']), cumm_income_dist_classes )
cumm_income_dist_classes_fmt = f"{cumm_income_dist_classif}[{','.join(cumm_income_dist_classes)}]"
#list(cumm_income_dist_classes)
#cumm_income_dist_classes_fmt

In [169]:
var_query_url = variables_url( cumm_income_dist_aggr, [cumm_income_dist_var], [cumm_income_dist_classes_fmt])
cumm_income_dist_response = query_ibge(var_query_url, http_fn )

In [170]:
cumm_income_dist_results = cumm_income_dist_response[0]['resultados']
cumm_income_dist_serii = map( lambda x: x['series'][0]['serie'], cumm_income_dist_results )
cumm_income_dist_serii = list(cumm_income_dist_serii)

In [174]:
cumm_income_dist_data = list(map(series_to_row_fn, cumm_income_dist_serii))
#cumm_income_dist_data = numpy.matrix(cumm_income_dist_data).transpose()
#cumm_income_dist_data = pandas.DataFrame( cumm_income_dist_data , index=years_queried, columns=income_bound_columns )
#cumm_income_dist_data

[[0.3, 0.2, 0.2, 0.2, 0.2, 0.2],
 [1.0, 0.9, 0.8, 0.8, 0.8, 0.9],
 [3.4, 3.1, 3.0, 2.9, 2.9, 3.4],
 [6.7, 6.3, 6.2, 6.0, 6.0, 6.7],
 [11.1, 10.5, 10.4, 10.1, 10.1, 11.2],
 [16.6, 15.8, 15.7, 15.4, 15.5, 16.8],
 [23.5, 22.8, 22.7, 22.2, 22.3, 23.8],
 [32.0, 30.9, 30.9, 30.4, 30.6, 32.2],
 [42.9, 41.8, 41.8, 41.2, 41.5, 43.0],
 [58.7, 57.6, 57.5, 56.9, 57.2, 58.4],
 [71.0, 70.1, 69.8, 69.4, 69.4, 70.5],
 [88.7, 88.4, 88.3, 87.8, 87.6, 88.2]]

In [ ]:
income_per_class_metadata = query_ibge( metadata_url(avg_income_per_class_aggr), http_fn )

In [ ]:
X = income_percentiles
widths = numpy.array([-5,  -5, -10, -10, -10, -10, -10, -10, -10, - 10,  -5,  -4,   -1])
Y = numpy.array(income_dist)/(-widths)

fig, ax = plot.subplots()
ax.bar(X, Y, align='edge', width=widths)

NameError: name 'income_dist' is not defined